# Interactive PursuedPyBear -- A minimal tutorial

Welcome to the interactive PursuedPyBear tutorial,
where we will learn to make games bit-by-bit.
PursuedPyBear (PPB) is designed to have little
unneeded code,
sometimes called
"boilerplate".
However, to tie together Jupyter and PPB,
we have to have a few uninteresting lines.

Bear, um, with us.
The following lines are the only boilerplate,
just execute the cell
(by clicking CTRL-Enter)
and move on.

In [1]:
import asyncio
from twisted.internet import asyncioreactor
asyncioreactor.install(asyncio.get_event_loop())
from twisted.internet import reactor
import ppb
import txppb
__file__ = None

pygame 1.9.6
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
class Penguin(ppb.Sprite):
    direction = ppb.Vector(0, 0)

In [3]:
def setup(scene):
    global SCENE
    SCENE = scene

In [16]:
d = txppb.run(setup, reactor)
d.addBoth(print)

In [17]:
SCENE.add(Penguin(pos=(0,0)))

In [6]:
from ppb import keycodes

DIRECTIONS = {keycodes.Left: ppb.Vector(-1,0), keycodes.Right: ppb.Vector(1,0),
              keycodes.Up: ppb.Vector(0, 1), keycodes.Down: ppb.Vector(0, -1)}

def Penguin_on_key_pressed(self, key_event, signal):
    self.direction = DIRECTIONS.get(key_event.key, ppb.Vector(0, 0))    

def Penguin_on_key_released(self, key_event, signal):
    if key_event.key in DIRECTIONS:
        self.direction = ppb.Vector(0, 0)
    
def Penguin_on_update(self, update_event, signal):
    self.position += update_event.time_delta * self.direction
 
Penguin.on_update = Penguin_on_update
Penguin.on_key_pressed = Penguin_on_key_pressed
Penguin.on_key_released = Penguin_on_key_released

In [7]:
class Fish(ppb.Sprite):
    pass

In [8]:
SCENE.add(Fish(pos=(1.5, 1)))

In [9]:
def Fish_on_update(self, update_event,signal):
    penguin, = update_event.scene.get(kind=Penguin)
    if (max(penguin.bottom, self.bottom) <= min(penguin.top, self.top) and
        max(penguin.left, self.left) <= min(penguin.right, self.right)):
        update_event.scene.remove(self)
Fish.on_update = Fish_on_update

In [10]:
SCENE.add(Fish(pos=(-2, -2)))

In [14]:
SCENE.add(Fish(pos=(0, 1)))

In [15]:
SCENE.add(Fish(pos=(0, 3)))

In [16]:
SCENE.add(Fish(pos=(1, 3)))

In [17]:
SCENE.add(Fish(pos=(0, 1)))

In [11]:
for i in range(5):
    SCENE.add(Fish(pos=(2, i/2)))

In [12]:
class OrangeBall(ppb.Sprite):
    pass

In [22]:
SCENE.add(OrangeBall(pos=(0,0)))

None


In [20]:
def smooth_step(t):
    return t * t * (3 - 2 * t)

In [18]:
import random
OrangeBall.is_moving = False
def OrangeBall_on_update(self, update_event,signal):
    if not self.is_moving:
        penguin, = update_event.scene.get(kind=Penguin)
        if (max(penguin.bottom, self.bottom) <= min(penguin.top, self.top) and
            max(penguin.left, self.left) <= min(penguin.right, self.right)):
            direction = (self.position - penguin.position).normalize()
            if direction == ppb.Vector(0, 0):
                direction = ppb.Vector(random.randrange(-1, 1), random.randrange(-1, 1)).normalize()
            self.target_position = self.position + direction
            self.original_position = self.position
            self.time_passed = 0
            self.is_moving = True
    else:
        self.time_passed += update_event.time_delta
        if self.time_passed >= 1:
            self.position = self.target_position
            self.is_moving = False
            return
        t = smooth_step(self.time_passed)
        self.position = (1-t) * self.original_position + t * self.target_position
OrangeBall.on_update = OrangeBall_on_update